In [ ]:
# mike babb
# find all anagram pairs

In [ ]:
import datetime
import os
import sqlite3

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from part_00_process_functions import build_db_conn, query_db

In [ ]:
sql = 'select * from words_v1;'

In [ ]:
db_path = '/project/finding_anagrams/db'
db_name = 'words.db'

In [ ]:
df = query_db(sql = sql, db_path = db_path, db_name = db_name)

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
# create the hash of the sorted letter group

In [ ]:
df['lcase_sorted'] = df['lcase'].map(lambda x: ''.join(sorted(x)))

In [ ]:
df['lcase_s_hash'] = df['lcase_sorted'].map(hash)

In [ ]:
df.head()

In [ ]:
df.loc[df['lcase']=='time']

In [ ]:
# find the complete anagrams
# anagram: nag, a, ram

In [ ]:
# let's assign a prime number to each letter
# calculate the first 26 prime numbers
# https://web.nmsu.edu/~pbaggett/Lessons/primeNumbers/primeNumbers.html
prime_list = []
curr_value = 2
while len(prime_list) < 26:
    is_prime = True
    cut_off = int(np.ceil(np.sqrt(curr_value)))
    cut_off += 1
    #print(curr_value, cut_off)
    for i in range(2, cut_off):        
        if curr_value != i: 
            if curr_value % i == 0 :
                is_prime = False
    
    if is_prime:
        prime_list.append(curr_value)
    
    curr_value += 1
    

In [ ]:
# letter list
letter_list = sorted(df['first_letter'].unique().tolist())

In [ ]:
letter_prime_dict = {x:y for x,y in zip(letter_list, prime_list)}

In [ ]:
def score_word(x):
    running_total = int(0)
    for curr_char in x:
        running_total += letter_prime_dict[curr_char]
    return running_total

In [ ]:
df['l_score'] = df['lcase'].map(score_word)

In [ ]:
# basically, find all of the valid permutations

In [ ]:
focal_word = 'formaldehydesulphoxylate'
#focal_word = 'anagram'

In [ ]:
fw_df = df.loc[df['word'] == focal_word, ]

In [ ]:
fw_id = fw_df['word_id'].iloc[0]

In [ ]:
fw_n_char = fw_df['n_chars'].iloc[0]

In [ ]:
fw_hash = fw_df['lcase_s_hash'].iloc[0]

In [ ]:
fw_sum = fw_df['l_score'].iloc[0]

In [ ]:
fw_sum

In [ ]:
# to words
sql = 'select distinct * from anagrams where from_word_id = ' + str(fw_id)

In [ ]:
cc_df = query_db(sql = sql, db_path = db_path, db_name = db_name)

In [ ]:
len(cc_df['to_word_id'].unique())

In [ ]:
# we need to use all of the letters
# how many ways can we group combinations of the to_words,
# that do not exceed the total number of characters in the anagram word

In [ ]:
df.head()

In [ ]:
temp_df = df[['word_id', 'lcase', 'lcase_s_hash', 'n_chars', 'l_score']]

In [ ]:
cc_df = pd.merge(left = cc_df, right = temp_df, left_on = 'to_word_id', right_on = 'word_id')

In [ ]:
#grand_list = []
def call_do_it(num_list, total_sum):
    combo_list = []
    grand_list = []    
    
    def do_it(num_list, total_sum, curr_sum = 0, curr_start = 0, combo_list = [], grand_list = []):                
                            
        if(total_sum == curr_sum):
            #print(combo_list)
            # so, they reason why it needs to be copied is that combo_list changes scope
            # recursively. Once all items are removed, all items are removed everywhere.
            # hence, the empty lists
            grand_list.append(combo_list.copy())
            if(len(grand_list) % 10000 == 0):
                print(combo_list[-1])
            #print(grand_list)
            #print(combo_list)
        else:        
            prev_element = -1             
            for i in range(curr_start, len(num_list)):
                if(prev_element != num_list[i]):
                    if(curr_sum + num_list[i] > total_sum): #array is sorted, no need to check further                    
                        break           
                    combo_list.append(num_list[i])            
                    do_it(num_list=num_list, total_sum=total_sum,
                          curr_sum = curr_sum + num_list[i], curr_start = i + 1,
                          combo_list = combo_list,
                          grand_list = grand_list)
                    #combinationList.remove(combinationList.size() - 1);
                    #print(combo_list)
                    #print(len(combo_list) - 1)
                    del combo_list[(len(combo_list) - 1)]
                    #prevElement = arrA[i];
                    prev_element = num_list[i]
        return grand_list                    
    output_list = do_it(num_list = num_list, total_sum = total_sum)
    return output_list

In [ ]:
num_list = sorted(cc_df['l_score'].tolist())

In [ ]:
my_list = [6, 2, 7, 8, 2, 4, 1, 3, 7, 5]

In [ ]:
my_list = sorted(my_list)

In [ ]:
my_list

In [ ]:
s_time = datetime.datetime.now()
output = call_do_it(num_list = my_list, total_sum = 8)
e_time = datetime.datetime.now()
p_time = e_time - s_time
p_time = p_time.total_seconds()
print(p_time)
print(len(output))

In [ ]:
output

In [ ]:
df = df.sort_values(by = 'l_score', ascending=False)

In [ ]:
df.head()

In [ ]:
len(cc_df)

In [ ]:
s_time = datetime.datetime.now()
output = call_do_it(num_list = num_list, total_sum = fw_sum)
e_time = datetime.datetime.now()
p_time = e_time - s_time
p_time = p_time.total_seconds()
print(p_time)
print(len(output))

In [ ]:
output = do_it(num_list = my_list, total_sum = 8, curr_start=0, combo_list=[], grand_list = [])

In [ ]:
output = do_it(num_list = num_list, total_sum = fw_sum)

In [ ]:
len(output)

In [ ]:
other_words = cc_df['lcase'].tolist()

In [ ]:
output_pairs = []
for ow in other_words:
    ow_length = len(ow)
    curr_other_words = cc_df.loc[(cc_df['lcase'] != ow) & \
                                 (cc_df['n_chars'] <= (fw_n_char - ow_length)), 'lcase']
    print(ow, len(curr_other_words))
    

In [ ]:
sorted(focal_word)

In [ ]:
n_char_list = sorted(cc_df['n_chars'].unique().tolist())

In [ ]:
n_char_list

In [ ]:
# use a simple loop to discover the partitions
part_pairs = []

In [ ]:
# find all of the ways to sum to 7

In [ ]:
from itertools import combinations, product, combinations_with_replacement

In [ ]:
# theoretically, we're only gonna have to do this once for each count of letters

In [ ]:
# now, let's score each word by assigning each letter a prime number and adding up the value

In [ ]:
letter_prime_dict

In [ ]:
df.head()

In [ ]:
testo = df.loc[df['lcase']=='anagram', ]

In [ ]:
testo.head()

In [ ]:
# n character sets
n_sets = [x for x in combinations_with_replacement(n_char_list, r = fw_n_char) if sum(x) == fw_n_char]

In [ ]:
n_sets

In [ ]:
len(list(product([1,2,3,4], repeat = 7)))

In [ ]:
output_list = []
for i in n_char_list:
    n_sets = [x for x in combinations_with_replacement(n_char_list, r = i) if sum(x) == fw_n_char]
    output_list.extend(n_sets)
    
        

In [ ]:
len(output_list)

In [ ]:
output_list

In [ ]:
s_time = datetime.datetime.now()
pair_list = []
count_list = 0
for curr_n_char_group in output_list:
    print(curr_n_char_group)
    curr_word_list = []
    for i_char in curr_n_char_group:
        print(i_char)
        curr_to_word_list = cc_df.loc[cc_df['n_chars']==i_char, 'lcase'].tolist()
        #curr_word_list.append(curr_to_word_list)
        curr_word_list.extend(curr_to_word_list)
        print(len(curr_word_list))
        temp_output = []
        for curr_letter_tuple in combinations(curr_word_list, r = len(curr_n_char_group)):
            curr_flat = ''.join(sorted(''.join(curr_letter_tuple)))
            #print(curr_flat)
            curr_hash = hash(curr_flat)
            if(curr_hash == fw_hash):
                count_list += 1
                pair_list.append(curr_letter_tuple)
                
            
            
    
    
    
    
    
    

In [ ]:
pair_list[:5]

In [ ]:
len(pair_list)